<a href="https://colab.research.google.com/github/harbidel/Age-Prediction-with-BMI-and-Face-Image/blob/main/BMI_%26_Face_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input


In [ ]:
# Load the dataset
data_url = "data.csv"
data = pd.read_csv(data_url)

In [ ]:
# Preprocess images and create image arrays
image_size = (128, 128)
images = []
for url in data['FACE IMAGE UPLOAD']:
    img = load_img(url, target_size=image_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    images.append(img_array)
images = np.array(images)

In [ ]:
# Normalize other features
bmi_data = data['BMI'].values.reshape(-1, 1)
age_data = data['Age'].values

In [ ]:
# Split the data into train, validation, and test sets
num_samples = len(data)
train_split = int(0.7 * num_samples)
val_split = int(0.15 * num_samples)
train_bmi, val_bmi, test_bmi = bmi_data[:train_split], bmi_data[train_split:train_split+val_split], bmi_data[train_split+val_split:]
train_images, val_images, test_images = images[:train_split], images[train_split:train_split+val_split], images[train_split+val_split:]
train_age, val_age, test_age = age_data[:train_split], age_data[train_split:train_split+val_split], age_data[train_split+val_split:]


In [ ]:
# BMI branch
bmi_input = Input(shape=(1,), name='bmi_input')
bmi_output = Dense(16, activation='relu')(bmi_input)


In [ ]:
# Image branch
image_input = Input(shape=image_size + (3,), name='image_input')
image_output = Conv2D(32, (3, 3), activation='relu')(image_input)
image_output = MaxPooling2D((2, 2))(image_output)
image_output = Flatten()(image_output)


In [ ]:
# Concatenate both branches
concatenated = Concatenate()([bmi_output, image_output])


In [ ]:
# Fully connected layers for fusion and age prediction
fusion_layer = Dense(64, activation='relu')(concatenated)
output_layer = Dense(1, activation='linear', name='output')(fusion_layer)  # Linear activation for age prediction


In [ ]:
# Create the model
model = Model(inputs=[bmi_input, image_input], outputs=output_layer)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model
model.fit({'bmi_input': train_bmi, 'image_input': train_images}, train_age, epochs=10, batch_size=32,
          validation_data=({'bmi_input': val_bmi, 'image_input': val_images}, val_age))


In [ ]:
# Evaluate the model
test_loss = model.evaluate({'bmi_input': test_bmi, 'image_input': test_images}, test_age)
print("Test Loss:", test_loss)
